In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import numba

In [2]:
cm = pd.read_csv('Movie_data_cleaned.csv')
cm['genres'] = cm.genres.str.split('/')
cm['tags'] = cm.tags.str.split('\|\|')
mymv = set(cm.movieId)

In [13]:
cm

,movieId,imdbId,title,language,year,director,tags,cast,genres,rating
0,180067,2371383,Louis Theroux: Extreme Love - Dementia,English,2012,Dan Child,[old age/phoenix arizona/senility/retirement h...,Louis Theroux,[Documentary],7.9
1,180075,6108612,The Connected Universe,English,2016,Malcom Carter,[pseudo science],Dasha German/Nassim Haramein/Patrick Stewart,[Documentary],7.6
2,180077,196699,The Lifestyle,English,1999,David Schisgall,[barbecue/old woman/old man/male nudity/female...,"e Amore"", /,",[Documentary],5.7
3,180079,838233,A Thousand Years of Good Prayers,English,2007,Wayne Wang,[language barrier/chinese american/boyfriend g...,Henry O/Pasha D. Lychnikoff/Feihong Yu/Vida Gh...,"[Romance, Drama]",6.7
4,180083,3496526,Geographically Desirable,English,2015,Mike Kravinsky,[dog],Blair Bowers/Andrew Agner-Nichols/Nic Detorie/...,"[Romance, Comedy]",5.4
...,...,...,...,...,...,...,...,...,...,...
23838,209069,272125,Snapshots,English,2002,Rudolf van den Berg,[briefcase of money/tent/mispronounced french/...,Burt Reynolds/Julie Christie/Carmen Chaplin/An...,[Romance],5.9
23839,209073,4015996,The Arbalest,English,2016,Adam Pinney,[obsession],Mike Brune/Tallie Medel/Matthew Stanton/Felice...,[Drama],5.6
23840,209085,10768348,The Mistletoe Secret,English,2019,Terry Ingram,[christmas],Kellie Pickler/Tyler Hynes/Christopher Russell...,"[Romance, Drama]",6.4
23841,209129,2361219,Destination Titan,English,2011,Stephen Slater,[space exploration/gravity assist/saturn the p...,John Zarnecki/Ralph Lorenz/Mark Leese/Patrick ...,[Documentary],7.8


In [ ]:
s = set()
for it in cm['rating']:
    s |= set([it])

In [4]:
with open('movie-id-map.pkl', 'rb') as f:
    mv2id = pickle.load(f)
with open('movie-id-map2.pkl', 'rb') as f:
    mv2id2 = pickle.load(f)

In [23]:
mymv = list(mymv)
mv2id = {mymv[i]: i for i in range(len(mymv))}
with open('movie-id-map.pkl', 'wb') as of:
    pickle.dump(mv2id, of)

In [4]:
def calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='f4')
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        for i in it:
            psim[p2id[i], mv2id2[mv]] += 1
    print('done psim')
    
    msim = np.zeros((len(mymv), len(mymv)), dtype='f4')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    return msim

In [ ]:
np.savez_compressed('director-similarity.npz', calculate_similarity('director'))

In [ ]:
def big_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    f = h5py.File('temp.hdf5', 'w')
    psim = f.create_dataset(cname+'-similarity', (len(dist), len(mymv)), dtype='f2')
    i = 0
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        psim[np.sort(np.array([p2id[i] for i in it])), mv2id[mv]] = 1
        i += 1
        if i%1000 == 0:
            print(f'psim {i}')
    print('done psim')
    f.close()
    return
    final = h5py.File('similarities.hdf5', 'w')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, i][:, np.newaxis]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    f.close()
    final.close()

In [ ]:
np.savez_compressed('genre-similarity.npz', calculate_similarity('genres'))

In [8]:
def rating_similarity():
    rsim = np.zeros((len(mymv), len(mymv)), dtype='f4')
    i = 0
    for mv, rt in cm[['movieId', 'rating']].itertuples(index=False):
        rsim[mv2id2[mv], :] += rt
        rsim[:, mv2id2[mv]] -= rt
        i += 1
        if i%1000 == 0:
            print('done', i)
    return rsim

In [ ]:
np.savez_compressed('rating-similarity.npz', rating_similarity())

In [ ]:
final['cast-similarity']

In [10]:
def half_similarity(cname: str):
    print('s')
    rf = h5py.File('/media/aayush/Acer/UbuntuStuff/temp.hdf5', 'r')
    print('l1')
    psim = rf[cname+'-similarity']
    print('l2')
    final = h5py.File('/media/aayush/Acer/UbuntuStuff/similarities.hdf5', 'w')
    print('l3')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    print('l4')
    for i in range(len(mymv)-1):
        print('start', i)
        row = rf[cname+'-similarity'][:, i]
        for j in range(i+1, len(mymv)):
            print('mid', j)
            msim[i, j] = np.linalg.norm(row - rf[cname+'-similarity'][:, j])
    rf.close()
    final.close()

In [ ]:
big_calculate_similarity('cast')


In [ ]:
half_similarity('cast')

In [5]:
def other_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname].dropna():
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='f4')
    for mv, it in cm[['movieId', cname]].dropna().itertuples(index=False):
        psim[[p2id[i] for i in it], mv2id2[mv]] += 1
    print('done psim')
    return psim
    msim = np.zeros((len(mymv), len(mymv)), dtype='f4')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        print(f'done{i}')
    return msim

In [12]:
@numba.jit(nopython=True, parallel=True)
def other_half_similarity(sim, nmv, start, end):
    print('Enter')
    msim = np.zeros(shape=(nmv, nmv), dtype=numba.float32)
    for i in range(start, end):
        col = sim[:, i]
        for j in range(i+1, nmv):
            msim[i, j] += np.linalg.norm(col-sim[:, j])
        print('done', i)
    return msim

In [8]:
psim = other_calculate_similarity('genres')

len 32
done psim


In [ ]:
other_half_similarity(psim, len(mymv), 0, 10)

In [19]:
def subset_similarity(cname: int, movies: list):
    subset = cm[cm.movieId.isin(set(movies))].dropna()
    dist = set()
    for it in subset[cname]:
        dist |= set(it)
    dist = list(dist)
    print('done set')
    p_to_ind = {dist[i]: i for i in range(len(dist))}
    movie_to_ind = {movies[i]: i for i in range(len(movies))}
    psim = np.zeros((len(dist), len(movies)), dtype='<f2')
    for mv, it in subset[['movieId', cname]].itertuples(index=False):
        psim[[p_to_ind[i] for i in it], movie_to_ind[mv]] += 1
    print('done psim')
    msim = np.zeros((len(movies), len(movies)), dtype='<f2')
    for i in range(len(movies)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
#         if i%100 == 0:
        print('msim', i)
    return msim

In [19]:
uids = pickle.load(open('most-active-users.pkl', 'rb'))

In [4]:

us = pd.read_csv('ml-25m/timeless_ratings.csv')
us = us[us.userId.isin(uids)]
us = us[us.movieId.isin(mv2id2)]

In [5]:
us.shape

(9017307, 3)

In [22]:
uids = np.array(list(uids))

In [7]:
rtu = np.array(us.userId)

In [8]:
rtm = np.array(us.movieId)

In [9]:
rtr = np.array(us.rating)

In [25]:
uids.shape

(20000,)

In [28]:
@numba.njit()
def makeumat(uu, nmv, start, end):
    msim = np.zeros((nmv, nmv))
    for i in range(start, end):
        col = uu[:, i]
        for j in range(i+1, nmv):
            msim[i, j] += np.linalg.norm(col-uu[:, j])
        if i%1000 == 0:
            print('done', i)
#     del uu
    return msim

In [ ]:
msim = makeumat(uu, len(mv2id2), 0, len(mv2id2))

In [27]:
#del us
uu = np.zeros((uids.shape[0], len(mv2id2)))
u2id = {uids[i]:i for i in range(len(uids))}
for i in range(len(rtu)):
    uu[u2id[rtu[i]], mv2id2[rtm[i]]] = rtr[i]

In [7]:
psim = other_calculate_similarity('tags')

len 24529
done psim


In [ ]:
# same process for genre-similarity.npz
with np.load('all_similarities/rating-similarity.npz') as npf:
    rs = npf['arr_0']
rs += 1
rs = np.reciprocal(rs)
np.savez_compressed('all_similarities/rating_sim_invscaled.npz', rs)
mm = np.memmap('all_similarities/rating-similarity.npy', 'w+', dtype='<f2', shape=(23843, 23843))
mm[:] = rs[:]
mm.flush()